In [15]:
import xarray as xr

from src.tools import RiversHandler, GridHandler, OutputHandler

In [16]:
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_version3.nc')
river_ds = xr.open_dataset('fram_data/norfjords_160m_river_v3++++.nc')